# Importing, Editing & Manipulating Configurations
* This scripts loads a mini scene defined in 'mini.g'.
* The 'watchFile' command allows you to edit the file in some other editor, while it is being redisplayed whenever you save.
* 'set/getJointState' shows how to access degrees-of-freedom of the scene
* 'eval' shows how to access other features, including its Jacobian

## Importing robot models
Let's first have a look at robot models that are already installed with rai. (They are defined in the `rai-robotModels` github repo, which is partly copied into the `.../site-packages/robotic/rai-robotModels` install path.)

In [1]:
from robotic import ry

In [2]:
print('The path where model files are pre-installed:\n', ry.raiPath(''))
# you could overwrite this with:
# ry.setRaiPath('your_path/rai-robotModels/')

The path where model files are pre-installed:
 /home/cornelius/Documents/.rai-venv/lib/python3.10/site-packages/robotic/rai-robotModels/


In [3]:
C = ry.Config()
C.addFile(ry.raiPath('panda/panda.g')) .setPosition([1.,.0,.0]) .setQuaternion([1,0,0,1])
C.addFile(ry.raiPath('panda/panda.g'), 'r_') .setPosition([1.5,.0,.0]) .setQuaternion([1,0,0,1])
C.addFile(ry.raiPath('pr2/pr2.g')) .setPosition([-1.,.0,.0])
C.addFile(ry.raiPath('baxter/baxter.g')) .setPosition([0,.0,1.]) .setQuaternion([1,0,0,1])
C.addFile(ry.raiPath('robotiq/robotiq.g')) .setParent(C.getFrame('panda_joint7')) .setRelativePosition([0., 0., .15])
C.view();

-- WARNING:graph.cpp:read:663(-1) nothing to delete with key 'visual'
-- WARNING:graph.cpp:edit:485(-1) no nodes edited! (from 'joint: { ctrl_H: 1 }')


The `addFile` returns the *first* frame defined in the file, which typically is the base frame of the whole robot. Therefore, by setting it's position, you can move the whole loaded robot.

Sometimes you want to add a model twice, but avoid duplicated frame names. With `addFile` you can specify a `prefix` string (here `r_`) which add that prefix to all frame names of the added robot. This is exemplified with the second panda added here.

The Robotiq example also shows that this base frame can be made a child of other frames -- attaching the robotiq to the panda arm. (Here in addition to the existing gripper, which would first have to be removed using `C.delFrame`.)

The following is a simple helper to articulare all dofs in the display (press `q` to interrupt):

In [4]:
C.animate()

In [5]:
del C

## Conversion from URDF
The python package should install a script `urdf2rai.py` in `.local/bin`. That converts a urdf-file to a .g-file. For instance, place a `ur10.urdf` and the `ur_description` folder into the same folder and run:
```
urdf2rai.py ur10.urdf > ur10_org.g
```
That should generate a `ur10_org.g` file that can be displayed with `kinEdit ur10_org.g`. Note, by default that **excludes** exporting collision shapes (as I typically replace them, e.g. by the convex hull of the visual shapes). The `-coll 1` option of the script should include also the collision shapes.

The `rai-robotModels` should have more info on conversion from URDF.

## Interactive Editing
When developing your own model (robot or scene), you could of course just use the python commands `addFrame` etc. But that might be cumbersome and not interactive enough. The .g-file format is fairly easy to edit. To help doing this more interactively, there is the `watchFile` method:

Open the file you want to edit (here `mini.g`) in any editor. At the same time, open it from within python and display it using `watchFile`. The key benefit is that `watchFile` reloads and re-displays the file whenever it is externally modified (it watches the file's inode). That enables interactive editing.

**TODO**: The following is still not crash-robust when readers throw syntax error. kinEdit better?

In [6]:
from robotic import ry

In [7]:
C = ry.Config()
C.addFile('mini.g')
C.view();

In [8]:
C.watchFile('mini.g')  # Use 'ESC' to close the configuration viewer

-- kin.cpp:watchFile:3510(0) reloading `mini.g' ... 
Configuration: q.N=1 #frames=3 #dofs=1 #shapes=3 #ucertainties=0 #proxies=0 #forces=0 #evals=152
-- kin.cpp:watchFile:3537(0) watching...
The file z.log.global was modified.


In [9]:
del C

## How to attach frames - faking grasps
Note, this is not real grasping. Just editing the kinematic tree in your configuration

In [10]:
from robotic import ry
import numpy as np
import time

In [11]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandasTable.g'))
C.view();

In [12]:
C.attach("l_gripper", "r_gripper")

In [13]:
#move a bit around

q = C.getJointState()

for t in range(30):
    q[0] = np.sin(t/10)
    
    C.setJointState(q)
    C.view()
    time.sleep(0.1)

In [14]:
del C

## Advanced: YAML and dict representations

In [15]:
import yaml

with open(ry.raiPath('panda/panda_clean.g'), 'r', encoding='utf-8') as fil:
    model = yaml.safe_load(fil)

print(model)

{'panda_link0': {}, 'panda_link0_0(panda_link0)': {'shape': 'mesh', 'color': [0.9, 0.9, 0.9], 'mesh': '<meshes/visual/link0.ply>', 'visual': True}, 'panda_joint1_origin(panda_link0)': {'Q': [0, 0, 0.333, 1, 0, 0, 0]}, 'panda_joint1(panda_joint1_origin)': {'joint': 'hingeZ', 'limits': [-2.8973, 2.8973, 2.175, -1, 87], 'ctrl_limits': [2.175, -1, 87]}, 'panda_link1(panda_joint1)': {}, 'panda_link1_0(panda_link1)': {'shape': 'mesh', 'color': [0.9, 0.9, 0.9], 'mesh': '<meshes/visual/link1.ply>', 'visual': True}, 'panda_joint2_origin(panda_link1)': {'Q': [0, 0, 0, 0.707107, -0.707107, 0, 0]}, 'panda_joint2(panda_joint2_origin)': {'joint': 'hingeZ', 'limits': [-1.7628, 1.7628, 2.175, -1, 87], 'ctrl_limits': [2.175, -1, 87]}, 'panda_link2(panda_joint2)': {}, 'panda_link2_0(panda_link2)': {'shape': 'mesh', 'color': [0.9, 0.9, 0.9], 'mesh': '<meshes/visual/link2.ply>', 'visual': True}, 'panda_joint3_origin(panda_link2)': {'Q': [0, -0.316, 0, 0.707107, 0.707107, 0, 0]}, 'panda_joint3(panda_joint3